In [1]:
# Create the Client
from radicalbit_platform_sdk.client import Client
base_url = "http://localhost:9000"
client = Client(base_url)
model = client.get_model(id="e02f2aed-cd29-4703-8faf-2dcab9fc668e")

In [2]:
ref = model.get_reference_datasets()[0]
cur1 = model.get_current_datasets()[0]

In [4]:
from radicalbit_platform_sdk.charts import RadicalbitChart, RbitChartData

RadicalbitChart().numerical_feature_chart(data=RbitChartData(
    model=model,
    reference=ref
))

